# Camera calibration

In [ ]:
import os
import glob
import cv2
import copy
import numpy as np
from utils import *
from utils.frame_extractor import frame_extractor

### Create calibration images

In [ ]:
data_folder = "../data/calibration"
frame_extractor(camera_index=0, width=1280, height=720, output_dir=data_folder)

#### Load images

In [ ]:
imgs_path = glob.glob(os.path.join(data_folder, '*.jpg'))
imgs = [cv2.imread(img_path) for img_path in imgs_path]

#### Detect and refine corners

In [ ]:
corners = [cv2.findChessboardCorners(imgs[i], (8,6)) for i in range(len(imgs))]
for i, foto in enumerate(corners):
    if not foto[0]:
        print(f"Corners on image {imgs_path[i]} couldn't be found")
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 27, 0.01)

imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [ ]:
imgs_copy = copy.deepcopy(imgs)
corners_drawn_imgs = [cv2.drawChessboardCorners(img_copy, (8,6), corner_r, True) for img_copy, corner, corner_r in zip(imgs_copy, corners, corners_refined) if corner[0]]
output_dir = "../results/calibration"
os.makedirs(output_dir, exist_ok=True)

for i,image in enumerate(corners_drawn_imgs):
    img_name = f"{i}"
    u.show_image(image, img_name)
    u.write_image(output_dir, img_name, image)

#### Obtain parameters

In [ ]:
def get_chessboard_points(chessboard_shape, dx, dy):
    puntos = []
    for j in range(chessboard_shape[1]):
        for i in range(chessboard_shape[0]):
            puntos.append([dx*i, dy*j, 0])
    return np.array(puntos, np.float32)

In [ ]:
chessboard_points = get_chessboard_points((8, 6), 27, 27)

In [ ]:
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)
print(imgs_copy[1].shape)
obj_points = [chessboard_points for _ in range(len(valid_corners))]
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, valid_corners, (1280, 720), None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))
print("Intrinsics:\n", intrinsics)
print("Extrinsics:\n", extrinsics)
# Para la imagen 1 la matriz extrinseca es extrinsics[0]
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)